### This code responsible for answering the first sub-research question

### Importing necessary libararies

In [ ]:
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer,util
from Utility.ThesisUtility import plotGraph

### constants

In [1]:
model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

goldStandardPath = "../CVS Files/Teachers prompt/Goldstandards.xlsx"

sentencesPath = "" ## the reference to the excel file where your sentneces are located with the corresponding.

### Reading the gold stanards and the sentences of a prompt

In [ ]:
df_ref = pd.read_excel(goldStandardPath) # reading gold standards
def_sentences = pd.read_excel(sentencesPath) # reading the sentences

listRef = df_ref['Gold standard'] # list of only the text of the gold standards
listSentences = def_sentences['text'] # list of only the tests  of the sentneces

listembReferences = []
corpus_embeddings =[]

# calculating the embeddings of all of the gold standards
for ref in listRef:
    listembReferences.append(model.encode(ref))

# calculating the embeddings of all of the sentences
for sentence in listSentences:
    corpus_embeddings.append(model.encode(sentence))

allCosinSimiliarityValues = [] # contains the value of the highest cosine similairty between a sentence and all gold standards
countMissed =[] # contains the number of ignored items for each threshold

thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95]

for emb in corpus_embeddings:
    listMax = []
    for embref in listembReferences:
        cos_sim = util.cos_sim(emb, embref)
        listMax.append(cos_sim)
    index_max = np.argmax(listMax) #the index  of the highest cosine simliarity with this sentnece
    allCosinSimiliarityValues.append(listMax[index_max]) # the value of the highest cosine similairty with this sentence

for threshold in thresholds:
    temp = 0
    for i in range(0,len(allCosinSimiliarityValues)):
        if (allCosinSimiliarityValues[i] <threshold): #threshold to know if this item does not match any of the references!!!
            temp = temp +1
            continue
    countMissed.append(temp)

#plotting the graph between the ignored items and the threshodsl
plotGraph(thresholds,countMissed)